# Object oriented programming



## What is object oriented programming?

Object oriented programming is a way of programming that was first introduced with java. With object OOP so called objects are introduced which create a logical and realistic way of groupping information. In python (and many other languages) objects are defined with classes:

In [ ]:
class Vehicle:
    def __init__(self):
        ...





## What are classes?
Classes are a logical scope of a program groupping variables and functions respectively.
### Why do we need scoping?
<img alt="Scoping example" src="scoping.png" title="Scoping example" width="800"/>

- helps the garbage collector
- Helps with variable naming
- Helps with unwanted access to functions

### Scoping in related to classes
Lets redefine the Vehicle class with a few attributes. Attributes are variables that are part of an object. We can specifically tell python that a variable is a property as well.


In [ ]:
class Vehicle:
    def __init__(self,type):
        self.type = type #creates an instance variabe
       
    def __repr__(self):
        return f"from type {self.type}"
    
car = Vehicle("car") # object containing type car
plane = Vehicle("plane") # object containing type plane
print(car)
print(plane)

If we would print the inner variable state we would get the same types. (__repr__ actually does the same thing, you can specify how an object is represented when you call print on it)

In [ ]:
print(car.type)

I mentioned that we can set properties in classes specifically, this is useful if we have a function to compute variables on the fly and you don't want separate getters and setters for that.

In [ ]:
class Vehicle:
    def __init__(self,type:str,top_speed:int):
        self.type = type #creates an instance variabe
        self.top_speed = top_speed 
    
    @property
    def is_fast(self):
        if self.top_speed > 100:
            return "very speed"
        else:
            return "not so speed"

    def __repr__(self):
        return f"from type {self.type}"

car = Vehicle("car",50) # object containing type car
plane = Vehicle("plane",500) # object containing type plane
print(f"The car is {car.is_fast}")
print(f"The plane is {plane.is_fast}")

## Inharitance
Lets say I have a few classes where the majority of the functionality is the same. In cases like this we use inharitance. 


In [ ]:
from abc import ABC
class Vehicle(ABC): # specifies an abstract class (these classes cant be instantiated)
    def go_forward(self):
        return "going forward"
        
class Car(Vehicle):
    
    def __init__(self):
        self.type = "car"
        
    def __repr__(self):
        return f"{self.type} is {self.go_forward()}"

class Plane(Vehicle):

    def __init__(self):
        self.type = "plane"

    def __repr__(self):
        return f"{self.type} is {self.go_forward()}"

car = Car()
print(car)
plane = Plane()
print(plane)


Even though in haritance can be useful it is strongly recommended to not use it, as it quickly adds a lot of depth to the code where it's hard to follow.
In most cases it is better to get away interfaces, in python there is no explicit interface type however there is `Protocol` class type instead. Protocols basically only define the skeleton of a class rather than do an actual inharitance. This also means they are not enforced during a runtime, however the typechecker can pick it up.

In [ ]:
from typing import Protocol

class IVehicle(Protocol):
    def go_forward(self):
        ...
    def go_backwards(self):
        ...
    
class Car:
    def go_forward(self):
        print("Car is going forward")
        
    def go_backwards(self):
        print("Car is going backwards")

class Plane:
    def go_forward(self):
        print("Plane is going forward")

    def go_backwards(self):
        print("Plane is going backwards")


I mentioned typing before and used it in a few examples as well. Typing basically allows us to enforce variable types during writing. However types don't affect runtime, they are not enforced

In [ ]:
def print_go_forward_vehicle(vehicle:Vehicle):
    vehicle.go_forward()
    
car = Car()
string:str = "some data"
print_go_forward_vehicle(car)
print_go_forward_vehicle(string)

### Default python datastructures

In [ ]:
l:list[int]  = [i for i in range(10)]
t:tuple[int,int] = (1,2)
d:dict[int:int] = dict([(i,i+1) for i in range(10)])
s:set[int] = set([i for i in range(10)])

print(l)
print(t)
print(d)
print(s)


## What is the workshop really about?
This is what OOP builds on inharitance and reducing dependent code with being as generic as possible. I could go over now the default design patterns however in python you likely won't use them very often as many of them are obsolete now. Rather I will focus on how can you implement functional python systems with as low coupling(inter code dependency) as possible

# The Pythonic programming
In python everything is an object (except from the primitive types), even functions. This basically means and going to be visible later in the workshop, python functions and classes are interchangeable in many ways, and depending on what the specific use case, it makes our choice a bit easier.

## Defining classes
Classes supposed to have relatively low "knowledge about the code base". This means that classes should focus on one specific task but for that very well. A class that has multiple responsibilities isn't very good as it introduces unwanted coupling and makes the class harder to maintain. So in general de default way to define classes is to devide them into categories:
- Action first
- Data first


In [ ]:

class Engine:
    def __init__(self,rpm,torque,fuel_consumption):
        self.rpm = rpm
        self.torque = torque
        self.fuel_consumption = fuel_consumption
            
class CarWithEngine:
    def __init__(self,engine:Engine):
        self.engine = engine
        
    def go_forward(self):
        print("speeding up")
        rpm_new = self.engine.rpm * 2
        fuel_new = self.engine.fuel_consumption * 2
        self.engine.rpm = rpm_new
        
    def print_engine_rpm(self):
        print(f"Engine is on {self.engine.rpm} rpm")
        
engine = Engine(1000,300,5)
car = CarWithEngine(engine)
car.print_engine_rpm()
car.go_forward()
car.print_engine_rpm()



## Strategy design pattern
Used by many specifically for code interchangeability and extension without the need to modify underlying code

In [ ]:
class Engine(Protocol):
    def get_fuel(self):
        ...
    def get_torque(self):
        ...
    
class HighPowerEngine:
    def __init__ (self):
        self.rpm = 1000
        self.torque = 1000
        self.fuel_consumption = 15
        
    def get_fuel(self):
        return self.fuel_consumption
    
    def get_torque(self):
        return self.torque

class LowPowerEngine:
    def __init__ (self):
        self.rpm = 1000
        self.torque = 500
        self.fuel_consumption = 7.5

    def get_fuel(self):
        return self.fuel_consumption

    def get_torque(self):
        return self.torque
    
class Truck:
    def __init__(self):
        self.engine = LowPowerEngine()
        
    def change_engine(self,new_engine:Engine):
        self.engine = new_engine
    
    def __repr__(self):
        return f"current available torque is {self.engine.torque} and consumption is {self.engine.fuel_consumption}"
    
    
truck = Truck()
print(truck)
high_power = HighPowerEngine()
truck.change_engine(high_power)
print(truck)

Implement a medium power engine with torque of 750 and fuel of 10

In [ ]:
#Your solution here
